In [ ]:
from pathlib import Path
import re

def read_dataset(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = raw_text.split('\n')[4:]
#     ids_docs = []
#     keywords_docs = []
#     texts_docs = []
#     labels_docs = []
    ids = []
    keywords = []
    texts = []
    labels = []
    for line in raw_docs:
        
#         for line in doc.split('\n'):
        id_, keyword, text, label = line.split('\t')[0],line.split('\t')[2],line.split('\t')[4],line.split('\t')[5]
        ids.append(id_)
        keywords.append(keyword)
        texts.append(text)
        if(label=='0' or label=='1'):
            labels.append(0)
        else:
            labels.append(1)
#         labels.append(label)
#         token_docs.append(tokens)
#         tag_docs.append(tags)

    return ids, keywords, texts, labels

In [ ]:
IDs,Keywords,Texts,Labels = read_dataset('dontpatronizeme_pcl.tsv')

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(Texts, Labels, test_size=.2)

In [ ]:
len(train_tags)

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)

In [ ]:
for i in range(len(train_encodings['input_ids'][0])):
    print(train_encodings['input_ids'][0][i],"---",train_texts[0].split(' ')[i])

In [ ]:
train_texts[0]

In [ ]:
len(train_encodings['input_ids']),len(train_tags),len(val_encodings['input_ids']),len(val_tags)

In [ ]:
class Classifier_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Classifier_Dataset(train_encodings, train_tags)
val_dataset = Classifier_Dataset(val_encodings, val_tags)

In [ ]:
train_dataset[0]

In [ ]:
import tensorflow as tf

In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

